In [97]:
import os, datetime

import numpy as np
import pandas as pd

In [43]:
data_dir = '../data/'

df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df_test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
df_revealed_test = pd.read_csv(os.path.join(data_dir, 'revealed_test.csv'))
df_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
df_census = pd.read_csv(os.path.join(data_dir, 'census_starter.csv'))

In [44]:
df_train.info() # 122,265 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122265 entries, 0 to 122264
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   row_id                 122265 non-null  object 
 1   cfips                  122265 non-null  int64  
 2   county                 122265 non-null  object 
 3   state                  122265 non-null  object 
 4   first_day_of_month     122265 non-null  object 
 5   microbusiness_density  122265 non-null  float64
 6   active                 122265 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 6.5+ MB


In [15]:
print(f'unique counties: {df_train.cfips.nunique()}')

unique counties: 3135


In [26]:
pd.options.display.float_format = '{:.2f}'.format
df_train[['microbusiness_density', 'active']].describe()

,microbusiness_density,active
count,122265.00,122265.00
mean,3.82,6442.86
std,4.99,33040.01
min,0.00,0.00
25%,1.64,145.00
50%,2.59,488.00
75%,4.52,2124.00
max,284.34,1167744.00


In [73]:
df_train.sort_values('microbusiness_density', ascending=False).head(5)

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
94382,46127_2019-10-01,46127,Union County,South Dakota,2019-10-01,284.34,32011
94380,46127_2019-08-01,46127,Union County,South Dakota,2019-08-01,277.54,31245
122030,56033_2022-10-01,56033,Sheridan County,Wyoming,2022-10-01,227.76,54509
68673,32510_2022-05-01,32510,Carson City,Nevada,2022-05-01,224.54,98716
68670,32510_2022-02-01,32510,Carson City,Nevada,2022-02-01,217.59,95660


In [70]:
df_train[df_train['cfips'] == 56033].head() # Sheridan County, Wyoming - increasing trend influenced by active

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
121992,56033_2019-08-01,56033,Sheridan County,Wyoming,2019-08-01,30.18,7065
121993,56033_2019-09-01,56033,Sheridan County,Wyoming,2019-09-01,31.71,7423
121994,56033_2019-10-01,56033,Sheridan County,Wyoming,2019-10-01,31.99,7488
121995,56033_2019-11-01,56033,Sheridan County,Wyoming,2019-11-01,32.78,7672
121996,56033_2019-12-01,56033,Sheridan County,Wyoming,2019-12-01,32.81,7679


In [66]:
df_train[df_train['cfips'] == 46127].head() # Union County, South Dakota

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
94380,46127_2019-08-01,46127,Union County,South Dakota,2019-08-01,277.54,31245
94381,46127_2019-09-01,46127,Union County,South Dakota,2019-09-01,5.88,662
94382,46127_2019-10-01,46127,Union County,South Dakota,2019-10-01,284.34,32011
94383,46127_2019-11-01,46127,Union County,South Dakota,2019-11-01,5.87,661
94384,46127_2019-12-01,46127,Union County,South Dakota,2019-12-01,5.86,660


In [63]:
df_train.sort_values('active', ascending=False).head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
7887,6037_2020-05-01,6037,Los Angeles County,California,2020-05-01,14.87,1167744
7886,6037_2020-04-01,6037,Los Angeles County,California,2020-04-01,14.79,1160868
7916,6037_2022-10-01,6037,Los Angeles County,California,2022-10-01,14.67,1153292
7913,6037_2022-07-01,6037,Los Angeles County,California,2022-07-01,14.66,1152842
7914,6037_2022-08-01,6037,Los Angeles County,California,2022-08-01,14.65,1151836


In [ ]:
# convert first_day_of_month from str to datetime
df_train.first_day_of_month = pd.to_datetime(df_train.first_day_of_month)
df_test.first_day_of_month = pd.to_datetime(df_test.first_day_of_month)
df_revealed_test.first_day_of_month = pd.to_datetime(df_revealed_test.first_day_of_month)

In [ ]:
strformat = '%Y-%m'
print(f'train period:		{df_train.first_day_of_month.min().strftime(strformat)} ~ {df_train.first_day_of_month.max().strftime(strformat)}')
print(f'test period: 		{df_test.first_day_of_month.min().strftime(strformat)} ~ {df_test.first_day_of_month.max().strftime(strformat)}')
print(f'revealed period: 	{df_revealed_test.first_day_of_month.min().strftime(strformat)} ~ {df_revealed_test.first_day_of_month.max().strftime(strformat)}')

train period:		2019-08 ~ 2022-10
test period: 		2022-11 ~ 2023-06
revealed period: 	2022-11 ~ 2022-12


In [87]:
df_train[df_train['cfips'] == 1001][['microbusiness_density', 'active']].tail()

,microbusiness_density,active
34,3.35,1422
35,3.44,1461
36,3.42,1455
37,3.44,1463
38,3.46,1472


In [158]:
from dateutil.relativedelta import relativedelta

pred_range = 2

df = df_train[df_train['cfips'] == 1001]
df_next_records = pd.DataFrame(index=[df.index.max() + month for month in range(1,pred_range+1)], columns=df.columns)
df = pd.concat([df, df_next_records], axis=0)
df[['microbusiness_density', 'active']].rolling(window=pred_range).mean().shift(pred_range).tail(pred_range)

/var/folders/jz/wsn3kwpx7cq0nsrxkk2g7nrr0000gn/T/ipykernel_58018/2208840458.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_next_records], axis=0)


,microbusiness_density,active
39,3.43,1459.00
40,3.45,1467.50


In [157]:
df_revealed_test[df_revealed_test['cfips'] == 1001].tail() #[['microbusiness_density', 'active']]

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2022-11-01,1001,Autauga County,Alabama,2022-11-01,3.44,1463
1,1001_2022-12-01,1001,Autauga County,Alabama,2022-12-01,3.47,1475


In [153]:
# SMAPE
def SMAPE(actuals, preds):
	smape = 0
	for actual, pred in zip(actuals, preds):
		smape += abs(actual-pred) / (abs(actual) + abs(pred)) * 2 * 100
	return smape / len(actuals)

# 예제 데이터 생성 (실제값과 예측값)
data = {
    'date': pd.date_range(start='2020-01-01', periods=10, freq='D'),
    'actual': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550],
    'forecast': [110, 140, 210, 260, 295, 340, 410, 460, 480, 570]
}
df = pd.DataFrame(data)

SMAPE(df['actual'], df['forecast']) # it's supposed to be 4.21 %

4.211920087419261

In [162]:
actuals = df[['microbusiness_density', 'active']].rolling(window=pred_range).mean().shift(pred_range).tail(pred_range)['microbusiness_density'].values
preds = df_revealed_test[df_revealed_test['cfips'] == 1001].tail()['microbusiness_density'].values

In [169]:
smape_value = SMAPE(actuals, preds).item()
smape_value

0.391775378913089